In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.manufacturer.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global api, list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_request, url_check = f'''https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/exhibitor/profile/zh-CN/{crawler_series['Url'].removeprefix('https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/').removesuffix('.html')}/AUTOMECHANIKASHANGHAI''', crawler_series['Url']

                resp = requests.get(url_request,
                                    headers=get_header(**{'Apikey': api}),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
                    print(f'fuck api {api}')
                    time.sleep(3)
                    with open('./api.txt', 'r') as api_txt:
                        api = api_txt.read()
                elif resp.status_code == 200:
                    dict_ = resp.json()

                    if dict_['success'] == True:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if dict_['result']['products']['countTotal'] == 0:
            list_dict_correct_temp = [{'Url': crawler_series['Url']}]
        else:
            list_dict_correct_temp = []
            for i, dict_product in enumerate(dict_['result']['products']['products']):
                list_dict_correct_temp.append({'Url': crawler_series['Url'],
                                               '[Product] No': i + 1,
                                               '[Product] Url': f'''{crawler_series['Url'].removesuffix('.html')}/{dict_product['id']}.html'''})

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend(list_dict_correct_temp)

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Url', '[Product] No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/2.product_url-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/product_url_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[True] - [请求9次] - [剩余299条] - [2024/11/18 15:02:07] - 6169. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhejiang-shbd-auto-parts-technology-co-ltd.html

[True] - [请求59次] - [剩余298条] - [2024/11/18 15:02:07] - 6119. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhejiang-outeli-auto-air-conditioner-parts-co-ltd.html

[True] - [请求2次] - [剩余297条] - [2024/11/18 15:02:08] - 6182. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhejiang-siquan-electronic-technology-co-ltd.html

[True] - [请求8次] - [剩余296条] - [2024/11/18 15:02:08] - 6181. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhejiang-sinhe-supply-chain-management-coltd.html

[True] - [请求51次] - [剩余295条] - [2024/11/18 15:02:08] - 6163. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/seineca-aut